# **LINEARISATION UNIQUE DES TRAFICS SUR BORDEAUX METROPOLE**
> Traitements à mettre en oeuvre

In [75]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
sys.path.append(r'C:\Users\martin.schoreisz\git\Lin_uniq_BdxM\Lin_uniq_BdxM\Traitements')
import Connexion_Transfert as ct
import geopandas as gp
import pandas as pd
import numpy as np
from shapely.geometry import MultiPoint, Polygon, box
import re
from difflib import get_close_matches,SequenceMatcher
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from datetime import datetime
from collections import Counter
from Outils import creer_graph, plus_proche_voisin,nb_noeud_unique_troncon_continu,verif_index

from Simplifier_Rdpt import creer_dico_noeud_rdpt,simplifier_noeud_rdpt,maj_graph_rdpt
import Estim_trafic as et
import Comptage as Cpt
import Affectation_pt_comptage as ap

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 20)

from Base_BdTopo.Import_outils import *
from Base_BdTopo.Rond_points import *
from Base_BdTopo.Regroupement_correspondance import *
from Base_BdTopo.Troncon_elementaire import *
from Base_BdTopo.Troncon_base import *
from Base_BdTopo.Gestion_2_chaussee import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1 Regrouper les troncons du filaire de voie
>Les troncons du filaire de voie sont regroupés d'abord uniquement avec les troncons de cat_rhv égal à 1,2,3,61,62,63, pour pouvoir réaliser des troncon de trafic homogènes tous reliés entre eux.
<br> Ensuite, un deuxième regroupement comprenant l'ensemble des troncons de cat_rhv permetd'affiner le diagnostic

> ### 1.1 Import du fichier final
le fichier shape est vérifié à la main car l'affectation automatique n'est pas 100% fiable

In [55]:
# Pour info : lire le graph issu de la methode decrite dessous
with ct.ConnexionBdd('local_otv') as c :
    graph_filaire_123 = gp.read_postgis('select * from linearisation_bm.graph_rhv_123', c.connexionPsy)
    graph_filaire_123_vertex = gp.read_postgis('select id,cnt,chk,ein,eout,the_geom as geom from linearisation_bm.graph_rhv_123_vertices_pgr', c.connexionPsy)
    graph_filaire = gp.read_postgis('select * from linearisation_bm.graph_rhv_complet', c.connexionPsy)

In [56]:
gdf_rhv_groupe_123=gp.read_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gdf_rhv_groupe_123_v2.shp').merge(
    graph_filaire_123[['ident','source','target']], on='ident', how='left')
gdf_rhv_groupe=gp.read_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\rhv_grp.shp').merge(
    graph_filaire[['ident','source','target']], on='ident', how='left')

>### 1.2 Méthode
 pour information

In [45]:
# 1. import du fichier de filaire de base
gdf_rhv=gp.read_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_source\Bdx-Metro\Filaire_voie\FV_TRONC_L.shp')
# 2. mise en forme
gdf_rhv.columns=[a.lower() for a in gdf_rhv.columns] #nom de colonne en minuscule
gdf_rhv.rename(columns={'gid':'id','nom_voie' : 'numero'},inplace=True)
gdf_rhv=gdf_rhv.loc[~gdf_rhv.id.isna()].copy()
gdf_rhv['id_ign']=gdf_rhv.ident.apply(lambda x : 'TRONROUT'+str(x))
gdf_rhv=gdf_rhv.loc[~gdf_rhv.cat_rhv.isin(['98','99'])].copy()
gdf_rhv['nature']=gdf_rhv.apply(lambda x : 'Route à 2 chaussées' if x['rgraph_dbl']==0 else 'Route à 1 chaussée', axis=1)
gdf_rhv['nature']=gdf_rhv.apply(lambda x : 'Bretelle' if x['cat_dig']=='8' else x['nature'], axis=1)
gdf_rhv['numero']=gdf_rhv.apply(lambda x : 'Bretelle '+x['numero'] if x['cat_dig']=='8' else x['numero'], axis=1)
gdf_rhv['sens']=gdf_rhv.apply(lambda x : 'Direct' if x['rgraph_dbl']==0 else 'Double', axis=1)
gdf_rhv['codevoie_d']='NR'
gdf_rhv['importance']=gdf_rhv['cat_dig']
gdf_rhv['id']=gdf_rhv.id.apply(lambda x : int(x))
#gdf_rhv.loc[gdf_rhv.nom_voie.isna()] #verif des nom_voie null : 2 lignes, cf traitements plus loins
#filtrer les voies cylcables et autres
gdf_rhv_filtre=gdf_rhv.loc[~gdf_rhv.cat_dig.isin(['6','7','9','10'])].copy()

# 3. creer le graph en bdd
#creer_graph(gdf_rhv_filtre.loc[gdf_rhv_filtre.cat_rhv.isin(['1','2','3','33','61','62','63'])],
            #'local_otv',schema='linearisation_bm',table='graph_rhv_123',table_vertex='graph_rhv_123_vertices_pgr')
creer_graph(gdf_rhv_filtre,'local_otv',schema='linearisation_bm',table='graph_rhv_complet',table_vertex='graph_rhv_complet_vertices_pgr')

In [258]:
# 4. initialiser les paramètre de regroupement
df=import_donnes_base('local_otv','linearisation_bm', 'graph_rhv_complet','graph_rhv_complet_vertices_pgr')
df2_chaussees=df.loc[df.nature.isin(['Autoroute', 'Quasi-autoroute', 'Route à 2 chaussées'])]
df_avec_rd_pt,carac_rd_pt,lign_entrant_rdpt=identifier_rd_pt(df)
df_lignes=df_avec_rd_pt.set_index('id_ign')#mettre l'id_ign en index
bretelle=df_avec_rd_pt.loc[df_avec_rd_pt['nature']=='Bretelle'].copy()
bretelle_tri=bretelle.loc[bretelle.length.sort_values(ascending=False).index.tolist()].id_ign.tolist()
sans_bretelle=df_avec_rd_pt.loc[df_avec_rd_pt['nature']!='Bretelle'].copy()
list_sans_bretelle=sans_bretelle.id_ign.tolist()
list_tri_longueur=sans_bretelle.loc[sans_bretelle.length.sort_values(ascending=False).index.tolist()].id_ign.tolist()
list_id_ign=bretelle_tri+list_tri_longueur

c:\users\martin.schoreisz\appdata\local\programs\python\python37\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)
c:\users\martin.schoreisz\appdata\local\programs\python\python37\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)
c:\users\martin.schoreisz\appdata\local\programs\python\python37\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


In [45]:
# 5. appel fonction de grouepement pour creer des idtronc pour uniquement voies de 1 à 3 et d'un autre coté pour toute les voies
df_affectation, dico_erreur, lignes_traitees, lignes_non_traitees=regrouper_troncon(
    list_id_ign, df_avec_rd_pt, carac_rd_pt,df2_chaussees,[])

# 6. mise en forme et export
gdf_rhv_groupe=gdf_rhv_filtre.merge(df_affectation, left_on='id_ign', right_on='id', how='left')
gdf_rhv_groupe.to_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\rhv_grp_v2.shp')
#gdf_rhv_groupe_123=gdf_rhv_filtre.merge(df_affectation, left_on='id_ign', right_on='id', how='left')
#gdf_rhv_groupe_123.to_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gdf_rhv_groupe_123.shp')

In [265]:
#pour test sur une ligne
df_affectation2, dico_erreur2, lignes_traitees2, lignes_non_traitees2=regrouper_troncon(
    ['TRONROUT9919'], df_avec_rd_pt, carac_rd_pt,df2_chaussees,[])

0 2019-12-12 11:48:54.415530 nb lignes traitees : 0
pas de parrallele trouvee pour les troncons ['TRONROUT9919']


## 1. MISE EN FORME DES COMPTAGES PONCTUELS
Pour ces comptages l'idée c'est de réussir à regrouper les comptages par un idtronc issu de la detection des troncon elementaire. ça a du sens car pour un comptage ponctuel normalement les 2 sens de cicru sont sur un troncon unique. DE plus, les ens circu, noms de voie et autres attributs permettant de regrouper les comptages sont de qualité trop variable pour s'appuyer uniquement dessus. et enfin, peu de 2*2 vois sont meusrés

In [7]:
affect_finale,cpt_pct_l93=Cpt.traitements_bdxm_pct(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_source\Bdx-Metro\comptages\base_comptages_routiers_finale.xlsx',gdf_rhv_groupe)

c:\users\martin.schoreisz\appdata\local\programs\python\python37\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


## 2. MISE EN FORME DES COMPTAGES PERMANENTS
Pour ces compteurs on va regrouper en séparant les siredo des boucles liées au système gertrude. Pour les siredo pas de pb, pour les autre sc'est plus compliqué car les références de localiusation varient parfois. Il y a donc une phase de regroupement manuel à la fin

> ### 2.1 Import du fichier final

In [8]:
#import fichier corrige_final
cpt_perm_final=gp.read_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\cpt_perm_groupe.shp')

> ### 2.1 Méthode
Pour info

In [5]:
#creer un gdf avec la fusion des points et des données de comptages non nulles
cpt_brut=pd.read_excel(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_source\Bdx-Metro\comptages\comptage_trafic_final.xls')
cpt_brut.columns=[a.lower() for a in cpt_brut.columns]
cpt_brut=cpt_brut.loc[~cpt_brut.mjo_val.isna()].copy()
pt_cpt_perm=gp.read_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_source\Bdx-Metro\comptages\PC_CAPTE_P.shp')
pt_cpt_perm.columns=[a.lower() for a in pt_cpt_perm.columns]
pt_cpt_perm=pt_cpt_perm.merge(cpt_brut, on='ident')
pt_cpt_perm['nom_voie']=pt_cpt_perm.nom_voie.apply(lambda x : re.sub(('é|è|ê'),'e',x.lower().replace('  ',' ')))#mise en forme nom de voie

#regrouper les comptages
#un compteur peut caracteriser 1 ligne, ou 2 compteur 1 ligne ou 2 compteurs 2 lignes, dc les compteurs ne vont pas toujours par deux

#regrouper geographiquement
pt_cpt_perm['x_l93']=pt_cpt_perm.geometry.apply(lambda x : x.x)
pt_cpt_perm['y_l93']=pt_cpt_perm.geometry.apply(lambda x : x.y)
data_test_clust=[[x, y] for x, y in zip(pt_cpt_perm.x_l93.tolist(), pt_cpt_perm.y_l93.tolist())]
db = DBSCAN(eps=200, min_samples=2).fit(data_test_clust)
labels = db.labels_
pt_cpt_perm['n_cluster']=labels

#correspondance nom_rue
#analyser les noms de commune : 
#grp_nom_voie.nom_voie.apply(lambda x : x.split()[0]).unique()#liste des permiers mots
#grp_nom_voie.loc[grp_nom_voie.nom_voie.apply(lambda x : x.split()[0]=='carbon')].nom_voie.unique() #test des valeusr
dico_commune={'le bouscat','le haillan','bordeaux','pessac','talence', 'merignac','st medard','carbon blanc',
       'begles', 'eysines','villenave d\'ornon','bruges','gradignan','bassens','cenon','floirac','pessa','lormont', 'artigues'}
dico_type_voie={'allee' : ['all.','allee', 'allees'],'avenue': ['av', 'av.', 'avenue'], 'boulevard' : ['blvd'],
               'bretelle' : 'bretelle', 'cours' : ['crs','cours'], 'cote' : ['côte'], 'place':['pl'], 'route':['route', 'rte'],'rue':['rue'],
               'voie':['voie']}
def decoupe_nom_voie(nom_voie) : 
    for c in dico_commune : 
        if nom_voie[:len(c)]==c :
            commune=nom_voie[:len(c)]
            nom_voie=nom_voie[len(c):].strip()
    for k, v in dico_type_voie.items() : 
        if any([a for a in nom_voie.split() if a in v])  : 
            caractere=[a for a in nom_voie.split() if a in v][0]
            type_voie=k
            nom_voie=' '.join([n for n in nom_voie.split() if n!=caractere])
            break
    else : type_voie=None
    for test in ['avant','apres'] :
        if test in nom_voie : 
            return nom_voie.split(test)[0].strip(), test,nom_voie.split(test)[1].strip(), commune, type_voie
    else : return nom_voie, nom_voie,nom_voie,commune,type_voie

pt_cpt_perm['nom_rue']=pt_cpt_perm.nom_voie.apply(lambda x : decoupe_nom_voie(x)[0])
pt_cpt_perm['localisant']=pt_cpt_perm.nom_voie.apply(lambda x : decoupe_nom_voie(x)[1])
pt_cpt_perm['rue_refer']=pt_cpt_perm.nom_voie.apply(lambda x : decoupe_nom_voie(x)[2])
pt_cpt_perm['commune']=pt_cpt_perm.nom_voie.apply(lambda x : decoupe_nom_voie(x)[3])
pt_cpt_perm['type_voie']=pt_cpt_perm.nom_voie.apply(lambda x : decoupe_nom_voie(x)[4])
pt_cpt_perm.to_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\pt_cpt_perm.shp')

>>#### 2.2.1 SIREDO

In [ ]:
siredo=pt_cpt_perm.loc[pt_cpt_perm['type']=='SIREDO'].copy()#isoler siredo
cross_join=siredo.assign(key=1).merge(siredo.assign(key=1), on='key').drop('key',axis=1)#calcul des distances
cross_join['distance']=cross_join.apply(lambda x : x['geometry_x'].distance(x['geometry_y']),axis=1)
cross_join_filtre=cross_join[[c for c in cross_join.columns if c[-2:]=='_x']+['distance','ident_y']].rename(columns=
              {c : c[:-2] for c in cross_join.columns if c[-2:]=='_x'}).copy()#filtre des attribut
cross_join_filtre=cross_join_filtre.loc[(cross_join_filtre['ident']!=cross_join_filtre['ident_y'])].copy()
siredo_proches=cross_join_filtre.loc[(cross_join_filtre.groupby('ident')['distance'].transform(min)==
                                      cross_join_filtre['distance'])].copy()#plus proche voisin
siredo_proches['id_grpsiredo']=siredo_proches.reset_index().index
id_siredo_proches=(pd.concat([siredo_proches[['ident','ident_y', 'id_grpsiredo']],
           siredo_proches[['ident_y','ident', 'id_grpsiredo']].rename(columns={
    'ident_y':'ident', 'ident':'ident_y'})],sort=False).sort_values('ident').
    reset_index().drop_duplicates('index').drop_duplicates(['ident','ident_y']).drop('index',axis=1))
cle_assoc_siredo=pd.concat([id_siredo_proches[['ident','id_grpsiredo']],id_siredo_proches[['ident_y','id_grpsiredo']].rename(columns={
    'ident_y':'ident'})]).sort_values('id_grpsiredo')

>>#### 2.2.2 Gertrude
> au final ce n'est pas très efficace, et un renseignement des regroupement a la ain aurait été plus rapide. il sera important si de nouvelles stations sont posées de fournir de suite un identifiant de regroupement

In [23]:
gertrude=pt_cpt_perm.loc[pt_cpt_perm['type']=='BOUCLE'].copy()#isoler les boucles
gertrude['x']=gertrude.geometry.apply(lambda x : x.x)
gertrude['y']=gertrude.geometry.apply(lambda x : x.y)

In [ ]:
#si c'est sur le mm idtronc on regroupe

#rappatrier le numero d'id_tronc pour chaque comptage
#trouver la distance min à chaque objet ligne"du rhv groupe
grp_troncon_temp=gertrude.copy()
grp_troncon_temp.geometry=grp_troncon_temp.buffer(20)#passer la geom en buffer
intersct_buff_20=gp.sjoin(grp_troncon_temp,gdf_rhv_groupe,how='left',op='intersects')
intersct_buff_20.geometry=gertrude.geometry#repasser la geom en point
intersct_buff_20=intersct_buff_20.merge(gdf_rhv_groupe[['ident','geometry']], left_on='ident_right', right_on='ident')
intersct_buff_20['dist_pt_ligne']=intersct_buff_20.apply(lambda x : x['geometry_x'].distance(x['geometry_y']), axis=1) #définir la disance entre les points et ligne
joint_dist_min=intersct_buff_20.loc[intersct_buff_20.groupby('ident_left')['dist_pt_ligne'].transform(min)==intersct_buff_20['dist_pt_ligne']][['ident_left','idtronc','numero']].rename(
    columns={'ident_left':'ident'}).copy()#ne garder que la ligne la plus proche
grp_troncon=gertrude.merge(joint_dist_min, on='ident',how='left')#df finale

#regrouper
grp_idtronc=grp_troncon.groupby('idtronc')['ident'].agg(lambda x : tuple(x))
grp_idtronc=grp_idtronc.loc[grp_idtronc.apply(lambda x : len(x)>1)].reset_index().copy()
grp_idtronc['id_grp']=grp_idtronc.ident.rank()
#mettre en forme
grp_idtronc=pd.DataFrame([(a,g) for i,g in zip(grp_idtronc.ident.tolist(),grp_idtronc.id_grp.tolist()) for a in i], columns=['ident','id_grp'])
grp_final=grp_idtronc.copy()

In [292]:
# trouver les points qui restent et préparer un regrouepement par commune, nom de rue, distance et se,s

#points qui restent
gertrude_etape2=gertrude.loc[~gertrude['ident'].isin(grp_idtronc.ident.tolist())].copy()
#comparer les noms
cross_join=gertrude_etape2[['gid','geometry','ident','nom_voie', 'sens_cir','commune', 'nom_rue', 'localisant','rue_refer','type_voie']].assign(key=1).merge(
    gertrude_etape2[['gid','geometry','ident','nom_voie', 'sens_cir','commune', 'nom_rue', 'localisant','rue_refer','type_voie']].assign(key=1), on='key').drop('key',axis=1)#calcul des distances
cross_join['distance']=cross_join.apply(lambda x : x['geometry_x'].distance(x['geometry_y']),axis=1)
cross_join['comp_nom_rue']=cross_join.apply(lambda x : SequenceMatcher(None,x['nom_rue_x'], x['nom_rue_y']).ratio(), axis=1)#affecter une note a cahque relation

#vérifier que les sens de cicru sont cohérents et ne garder que les lignes avec un sens ok, un nom de voie proche et une distance proche
gertrude_join=cross_join.loc[(cross_join['comp_nom_rue']>0.6) & (cross_join['distance']<1000) & (cross_join['commune_x']==cross_join['commune_y']) & 
                             (cross_join['type_voie_x']==cross_join['type_voie_y'])].copy()
#on ajoute un attribut de verif du sens
def verif_sens(s1,s2) : 
    for sens in [['Sens Sortant', 'Sens Entrant'],['Sens N > S', 'Sens S > N'], ['Sens Nord > Sud', 'Sens Sud > Nord'],['Nord > Sud', 'Sud > Nord'],
                ['Boulevards Nord > Sud', 'Boulevards Sud > Nord']] :
        if s1 in sens and s2 in sens : 
            if s1!=s2 : 
                return True
        else : 
            continue
    return False
gertrude_join['valid_sens']=gertrude_join.apply(lambda x : verif_sens(x['sens_cir_x'],x['sens_cir_y']),axis=1)  
gertrude_join['id_grp']=gertrude_join.sort_values('ident_x').ident_x.rank(method="min")
gertrude_grp=gertrude_join.loc[gertrude_join['valid_sens']][['ident_x','ident_y','distance', 'comp_nom_rue', 'valid_sens','id_grp']].sort_values('ident_x')

In [228]:
#obtenir une liste des ident qui se regroupent
list_id_grp=pd.concat([gertrude_grp[['ident_x','id_grp']],gertrude_grp[['ident_y','id_grp']].rename(columns={'ident_y' : 'ident_x'})]).drop_duplicates([
    'ident_x','id_grp']).groupby('id_grp').ident_x.agg(lambda x : set(sorted(tuple(x)))).tolist()
dico={}
for e,c in enumerate(gertrude_grp.ident_x.tolist()) : 
    for l in list_id_grp : 
        if c in l :
            #print('c dans l','c : ',c,'l :',l)
            if c not in dico.keys() :
                dico[c]=list(l)
            else : 
                dico[c]+=list(l)
list_id_grp_temp=set([ v for v in {k : tuple(set(sorted(dico[k]))) for k in dico.keys()}.values()])
list_id_grp_final=set(v for v in {c :b for c in gertrude_grp.ident_x.tolist() 
 for b in [a for a in list_id_grp_temp if c in a] 
 if len(b) == max([len(c) for c in [a for a in list_id_grp_temp if c in a]])}.values())

In [281]:
#affecter un identifiant
def groupe_pt_gertrude_etape2(df_local) : 
    """
    regrouper les points à partir d'une liste d'ident selon l'ordre d'apparition 
    """
    limite_box=MultiPoint([(a.x,a.y) for a in df_local.itertuples()]).bounds
    plg_long, plg_larg=limite_box[2]-limite_box[0],limite_box[3]-limite_box[1]
    if max(plg_long,plg_larg)==plg_long : 
        df_local['next_sens']=df_local.sort_values('x').sens_cir.shift(-1,fill_value='NC')
        df_local['prev_sens']=df_local.sort_values('x').sens_cir.shift(1,fill_value='NC')
    else : 
        df_local['next_sens']=df_local.sort_values('y').sens_cir.shift(-1,fill_value='NC')
        df_local['prev_sens']=df_local.sort_values('y').sens_cir.shift(1,fill_value='NC')
    df_local['sens_comp']=df_local.apply(lambda x : x.prev_sens if x.next_sens=='NC' else x.next_sens,axis=1)
    df_local['verif_sens']=df_local.apply(lambda x : verif_sens(x['sens_cir'],x['sens_comp']),axis=1)
    df_local['id_grp']=-99
    i=0
    while i < len(df_local)-1:
        if df_local.iloc[i].verif_sens : 
            df_local.loc[df_local.index.isin(df_local.iloc[i:i+2].index.tolist()),'id_grp']=i
            i+=2
        else : 
            i+=1
    return

# à partior de la liste des ident groupé : 
grp_final2=grp_final.copy()
# on selectionne ces points dans la df
increment=grp_final2.id_grp.max()
for l in list_id_grp_final : 
    #if 'Z8CT13' not in l : continue
    df_local=gertrude_etape2.loc[gertrude_etape2.ident.isin(l)].copy()
    if len(df_local)==1:
        continue
    if len(df_local)==2 : 
        if (gertrude_join.loc[(gertrude_join.ident_x.isin(l)) & (gertrude_join['ident_x'] != gertrude_join['ident_y'])].valid_sens).all() : 
            increment+=1
            df_local['id_grp']=increment
            grp_final2=pd.concat([grp_final2,df_local[['ident','id_grp']]],sort=False)
    else :
        groupe_pt_gertrude_etape2(df_local)
        increment+=grp_final2.id_grp.max()
        df_local.loc[df_local['id_grp']!=-99,'id_grp']=df_local['id_grp']+increment
        grp_final2=pd.concat([df_local[['ident','id_grp']],grp_final2],sort=False)

In [317]:
#rappatrier l'id_grp sur les données de base et export pour modif manuelle
gertrude.merge(grp_final2, on='ident',how='left').to_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gertrude.shp')

#import des modifs manuelles et mise en forme finale
gertrude_manu=gp.read_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gertrude.shp')
gertrude_manu.loc[(gertrude_manu.id_grp.isna()) | (gertrude_manu.id_grp==-99),'id_grp']=gertrude_manu.loc[(gertrude_manu.id_grp.isna()) | (gertrude_manu.id_grp==-99),'id_grp'].reset_index().index + gertrude_manu.id_grp.max()
gertrude_manu.loc[gertrude_manu.type_grp.isnull(),'type_grp']='auto'
#export pour dernière modif manuelle
gertrude_manu.to_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gertrude_final.shp')

#fichier final
gertrude_fin=gp.read_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gertrude_final.shp')
#concat avec les siredo
cle_assoc_siredo['id_grpsiredo']=cle_assoc_siredo['id_grpsiredo']+gertrude_fin.id_grp.max()
cpt_perm_final=pd.concat([gertrude_fin,siredo.merge(cle_assoc_siredo, on='ident').rename(columns={'id_grpsiredo':'id_grp'})],axis=0, sort=False)
cpt_perm_final.to_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\cpt_perm_groupe.shp')

# 3. Affecter du trafic au filaire de voie

> ### On commence par affecter le trafic aux troncons des catégories 1,2,3 à partirdes comptages permanents.<br> Pour le détail des fonctions cf module Affectation_pt_comptage

In [ ]:
gdf_rhv_cpt_perm_123,lgn_proche_perm=ap.affectation_cpt_perm(cpt_perm_final,gdf_rhv_groupe,gdf_rhv_groupe_123,10)
#ap.export_cpt_perm_linearises(gdf_rhv_cpt_perm_123,r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gdf_rhv_trafic_123_cpt_perm.shp')

> ### ensuite les comptages ponctuels

In [ ]:
#affectation des comptages ponctuels et exports
gdf_traf_tot=ap.affectation_cpt_ponct(cpt_pct_l93, affect_finale, lgn_proche_perm, gdf_rhv_groupe, 10, gdf_rhv_groupe_123, gdf_rhv_cpt_perm_123)
#ap.export_cpt_ponct_linearises(gdf_traf_tot,r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gdf_rhv_trafic_123_cpt_perm-ponct.shp')

## 2. MMM
En premier lieu il faut mettre à jour les attributs en renomant de façon explicite puis en faisant la somme des 2 sens de circulation pour les voies doucble sens

In [26]:
#creer graph et importer 
creer_graph(fichier_src_simpl, 'local_otv',id_name='NO', schema='public', table='mmm_graph', table_vertex='mmm_graph_vertices_pgr')

In [223]:
#importer le graph
with ct.ConnexionBdd('local_otv') as c :
    graph_mmm_filaire = gp.read_postgis('select * from linearisation_bm.mmm_graph', c.connexionPsy)
    graph_mmm_filaire_vertex = gp.read_postgis('select * from linearisation_bm.mmm_graph_vertices_pgr', c.connexionPsy)

In [22]:
#importer le fichier simplifie
mmm_simple=gp.read_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\MMM_simplifie.shp')
cle_mmm_rhv=gp.read_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\appariementV0.shp')

# Affecter du trafic aux voies non connues

In [23]:
gdf_rhv_cpt_perm_123.reset_index(inplace=True)

In [24]:
#SIMPLIFIER LES RONDS POINTS

#simplifier la topologie en affectant une seule valuer de noeud aux voie arrivant sur un rond point
#trouver les noeud faisant partie d'un rond point 
df=import_donnes_base('local_otv','linearisation_bm', 'graph_rhv_123','graph_rhv_123_vertices_pgr')
df2_chaussees=df.loc[df.nature.isin(['Autoroute', 'Quasi-autoroute', 'Route à 2 chaussées'])]
df_avec_rd_pt,carac_rd_pt,lign_entrant_rdpt=identifier_rd_pt(df)

lgn_rdpt=df_avec_rd_pt.loc[~df_avec_rd_pt.id_rdpt.isna()].copy()
dico_noeud=creer_dico_noeud_rdpt(lgn_rdpt)

c:\users\martin.schoreisz\appdata\local\programs\python\python37\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)
c:\users\martin.schoreisz\appdata\local\programs\python\python37\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


In [25]:
# àpartir du dico on va remplacer les valeurs de source ou target par la valeur de clé du dico. 
gdf_base=gdf_rhv_cpt_perm_123.copy()
gdf_rhv_rdpt_simple=gdf_base.loc[~gdf_base.index.isin(lgn_rdpt.ident.to_list())].copy()
#remplacement des sources et targets : 
simplifier_noeud_rdpt(gdf_rhv_rdpt_simple, dico_noeud)
#puis recalculer le count du nb de ligne par noeud (en otant les lignes qui font les rdpoints)
cnt_maj=maj_graph_rdpt(gdf_rhv_rdpt_simple)
noeuds_estim, noeuds_grp, df_noeuds=et.noeuds_estimables(gdf_rhv_rdpt_simple)

In [ ]:
#caracteriser les noeuds
# 1. trouver ceux estimables
noeuds_estim, noeuds_grp, df_noeuds=et.noeuds_estimables(gdf_rhv_rdpt_simple)
liste_noeud_estim=noeuds_estim.noeud.tolist()
liste_noeud_traite=[]
dico_erreur={}
while liste_noeud_estim : 
    # pour chaque noeuds : 
      #calcul des troncons arrivants
    num_noeud=liste_noeud_estim[0]
    print(num_noeud)
    gdf_rhv_rdpt_simple=verif_index(gdf_rhv_rdpt_simple,'idtronc')#Îsi idtronc est en index on le repasse dans les colonnes
    liste_noeud_traite.append(num_noeud)
    df_troncon_noeud=et.carac_troncon_noeud(gdf_rhv_rdpt_simple, gdf_base, graph_filaire_123_vertex,num_noeud,df_noeuds)
      #si tous les trafics du noeud sont déjà renseignés on passe
    if (df_troncon_noeud.tmjo_2_sens!=-99).all() : 
        continue
      #determiner si on est dans le cas d'une voie pouvant etre estimée par calcul (3 troncon dont 1 seul manquant) ou par analogie avec le MMM
    if et.type_estim(df_troncon_noeud)=='calcul_3_voies' :
        et.maj_trafic_3tronc(df_troncon_noeud)
        df_update_traf=df_troncon_noeud[['idtronc','tmjo_2_sens']].set_index('idtronc').drop_duplicates()
    else : #RETRAVAILLER : CA NE MARCHE PAS COMME PREVU
        matrice_rhv=et.matrice_troncon_noeud_rhv(df_troncon_noeud,lgn_rdpt) 
        try :
            joint_fv_mmm_e2=et.estim_mmm_jointure_voies(matrice_rhv,cle_mmm_rhv)
        except et.PasCorrespondanceError : 
            dico_erreur[num_noeud]='PasCorrespondanceError'
            noeuds_estim, noeuds_grp, df_noeuds=et.noeuds_estimables(gdf_rhv_rdpt_simple)
            liste_noeud_estim=[n for n in noeuds_estim.noeud.tolist() if n not in liste_noeud_traite]
            continue
        try :
            trafic_inconnus_prior_cat=et.isoler_trafic_inconnu(joint_fv_mmm_e2)
        except et.PasDeTraficInconnuError: 
            dico_erreur[num_noeud]='PasDeTraficInconnuError' 
            noeuds_estim, noeuds_grp, df_noeuds=et.noeuds_estimables(gdf_rhv_rdpt_simple)
            liste_noeud_estim=[n for n in noeuds_estim.noeud.tolist() if n not in liste_noeud_traite]
            continue
        try :
            trafc_rens=et.trafic_mmm(trafic_inconnus_prior_cat,mmm_simple,num_noeud, graph_filaire_123_vertex,gdf_rhv_rdpt_simple,cle_mmm_rhv)
            df_update_traf=et.calcul_trafic_rhv_depuisMMM(trafc_rens).set_index('idtronc').rename(columns={'tmjo_2_sens_extrapol':'tmjo_2_sens'})
        except et.PasDeTraficError as e: 
            dico_erreur[num_noeud]=e 
            noeuds_estim, noeuds_grp, df_noeuds=et.noeuds_estimables(gdf_rhv_rdpt_simple)
            liste_noeud_estim=[n for n in noeuds_estim.noeud.tolist() if n not in liste_noeud_traite]
            continue
      #mise à jour du fichier source
    #try : 
    gdf_rhv_rdpt_simple=gdf_rhv_rdpt_simple.set_index('idtronc')
    gdf_rhv_rdpt_simple.update(df_update_traf[['tmjo_2_sens']])
    gdf_rhv_rdpt_simple.reset_index(inplace=True)
    gdf_base=gdf_base.set_index('idtronc')
    gdf_base.update(df_update_traf[['tmjo_2_sens']])
    gdf_base.reset_index(inplace=True)
    #except ValueError as e : 
        #dico_erreur[num_noeud]=e
      #recherche noeud estimable
        #noeuds_estim, noeuds_grp, df_noeuds=et.noeuds_estimables(gdf_rhv_rdpt_simple)
        #liste_noeud_estim=[n for n in noeuds_estim.noeud.tolist() if n not in liste_noeud_traite]
        #continue
    noeuds_estim, noeuds_grp, df_noeuds=et.noeuds_estimables(gdf_rhv_rdpt_simple)
    liste_noeud_estim=[n for n in noeuds_estim.noeud.tolist() if n not in liste_noeud_traite]

gdf_base.reset_index()[['id_x', 'ident', 'domanial', 'groupe', 'cat_dig', 'cat_rhv', 'passage',
       'rggraph_nd', 'rggraph_na', 'rgraph_dbl', 'numero', 'cdate', 'mdate',
       'id_ign', 'nature', 'sens', 'codevoie_d', 'importance', 'id_y',
       'idtronc', 'geometry','tmjo_2_sens','type_cpt', 'id_cpt_2_sens']].to_file(
    r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gdf_rhv_trafic_123_extrapol.shp')

In [129]:
# pour chaque noeuds : 
  #calcul des troncons arrivants
num_noeud=3056
print(num_noeud)
df_troncon_noeud=et.carac_troncon_noeud(gdf_rhv_rdpt_simple.reset_index(), gdf_base, graph_filaire_123_vertex,num_noeud,df_noeuds)
  #si tous les trafics du noeud sont déjà renseignés on passe
if (df_troncon_noeud.tmjo_2_sens!=-99).all() : 
    print('on passe')
  #determiner si on est dans le cas d'une voie pouvant etre estimée par calcul (3 troncon dont 1 seul manquant) ou par analogie avec le MMM
if et.type_estim(df_troncon_noeud)=='calcul_3_voies' :
    et.maj_trafic_3tronc(df_troncon_noeud)
    df_update_traf=df_troncon_noeud[['idtronc','tmjo_2_sens']].set_index('idtronc').drop_duplicates()
else : #RETRAVAILLER : CA NE MARCHE PAS COMME PREVU
    matrice_rhv=et.matrice_troncon_noeud_rhv(df_troncon_noeud,lgn_rdpt) 
    joint_fv_mmm_e2=et.estim_mmm_jointure_voies(matrice_rhv,cle_mmm_rhv)
    trafic_inconnus_prior_cat=et.isoler_trafic_inconnu(joint_fv_mmm_e2)
    trafc_rens=et.trafic_mmm(trafic_inconnus_prior_cat,mmm_simple,num_noeud, graph_filaire_123_vertex,gdf_rhv_rdpt_simple.reset_index(),cle_mmm_rhv)
    df_update_traf=et.calcul_trafic_rhv_depuisMMM(trafc_rens).set_index('idtronc').rename(columns={'tmjo_2_sens_extrapol':'tmjo_2_sens'})
"""gdf_rhv_rdpt_simple=gdf_rhv_rdpt_simple.set_index('idtronc')
gdf_rhv_rdpt_simple.update(df_update_traf[['tmjo_2_sens']])
gdf_rhv_rdpt_simple.reset_index(inplace=True)
gdf_base=gdf_base.set_index('idtronc')
gdf_base.update(df_update_traf[['tmjo_2_sens']])
gdf_base.reset_index(inplace=True)"""

3056


"gdf_rhv_rdpt_simple=gdf_rhv_rdpt_simple.set_index('idtronc')\ngdf_rhv_rdpt_simple.update(df_update_traf[['tmjo_2_sens']])\ngdf_rhv_rdpt_simple.reset_index(inplace=True)\ngdf_base=gdf_base.set_index('idtronc')\ngdf_base.update(df_update_traf[['tmjo_2_sens']])\ngdf_base.reset_index(inplace=True)"

In [127]:
noeud=3056
def trafic_reference_mmm(tmjo_2_sens_x,tmjo_2_sens_y, df, idtronc_x,idtronc_y) : 
        """
        trouver le tmjo ode reference du mmm pour le calcul des trafic manquants,si plueisuers lignes possible
        in : 
           tmjo_2_sens_x : float : trafic issu du filaire prealablement renseigne
           tmjo_2_sens_y : float : trafic issu du filaire prealablement renseigne
           df : df des voies a renseignees
           idtronc : float : ifentifiant du troncon concerne
        """
        if tmjo_2_sens_y == -99 : 
            df_traf_ref=df.loc[(df['idtronc_y']==idtronc_y) | (df['idtronc_x']==idtronc_y) ].copy()
            df_traf_ref['traf_ref']=df_traf_ref.apply(lambda x : x['tmja_tv_x'] if x['tmjo_2_sens_y']==-99 else x['tmja_tv_y'],axis=1)
            return df_traf_ref.traf_ref.max()
        else : 
            df_traf_ref=df.loc[(df['idtronc_y']==idtronc_x) | (df['idtronc_x']==idtronc_x) ].copy()
            df_traf_ref['traf_ref']=df_traf_ref.apply(lambda x : x['tmja_tv_x'] if x['tmjo_2_sens_y']==-99 else x['tmja_tv_y'],axis=1)
            return df_traf_ref.traf_ref.max()
    
def ajout_tmja(idtronc, tmja_base, noeud) : 
    """
    Pour les voies du MMM qui sont représentées par 2 lignes il faut ajouter les 2 trafics pour le troncon
    """
    troncon=et.Troncon(gdf_rhv_rdpt_simple,idtronc)
    noeud_parrallele=et.trouver_noeud_parrallele(troncon,graph_filaire_123_vertex, noeud,100)
    #trouver les lignes du troncon qui intersectent ce noeud
    lign_fin=troncon.df_lign_fin_tronc.loc[(troncon.df_lign_fin_tronc['source']==noeud_parrallele) | (troncon.df_lign_fin_tronc['target']==noeud_parrallele)]
    #jointure avec les données MMM
    trafc_mmm_sup=lign_fin[['idtronc','ident']].merge(cle_mmm_rhv[['NO','ident']], on='ident').merge(mmm_simple[['NO','tmja_tv']], on='NO')
    if trafc_mmm_sup.empty : 
        return tmja_base
    else : 
        return tmja_base+trafc_mmm_sup.tmja_tv.sum()

def somme_traf(idtronc, traf_rens) : 
    """
    pour les voies du MMM représenté par une ligne qui se sépare en 2 dur la fin (rd point par exemple))
    """
    nb_value=list(set(traf_rens.loc[traf_rens['idtronc_x']==idtronc].tmja_tv_x.tolist()+traf_rens.loc[traf_rens['idtronc_y']==idtronc].tmja_tv_y.tolist()))
    if len(nb_value)>=2 : 
        return (sum(nb_value))
    else : return (nb_value[0])

#test si les troncon MMM identifiés ont bien un trafic dans le fichier mmm simplifie. si pas le cas corrige le fichier simplifie
troncon_mmm_sstraf=[a for a  in trafic_inconnus_prior_cat.NO_x.tolist()+trafic_inconnus_prior_cat.NO_y.tolist() if a not in mmm_simple.NO.tolist()]
if troncon_mmm_sstraf : 
    raise PasDeTraficError(troncon_mmm_sstraf)
traf_mmm=trafic_inconnus_prior_cat.merge(mmm_simple[['NO','tmja_tv']], left_on='NO_x', right_on='NO').drop('NO', axis=1).merge(
mmm_simple[['NO','tmja_tv']], left_on='NO_y', right_on='NO').drop('NO', axis=1)
#mise a jour des trafic mmm si 2*2voies MMM
if traf_mmm.apply(lambda x : ','.join([str(x['idtronc_x']),str(x['idtronc_y'])]),axis=1).nunique()!=len(traf_mmm) :
    traf_mmm['tmja_tv_x']=traf_mmm.apply(lambda x : ajout_tmja(x['idtronc_x'], x['tmja_tv_x'], noeud),axis=1)
    traf_mmm['tmja_tv_y']=traf_mmm.apply(lambda x : ajout_tmja(x['idtronc_y'], x['tmja_tv_y'], noeud),axis=1)
#ensuite on refait un test dans le cas par exemple des rd pt pour lesquels les voies se séparent à la fin
traf_mmm2=traf_mmm.copy()
traf_mmm2['tmja_tv_x']=traf_mmm2.apply(lambda x : somme_traf(x['idtronc_x'], traf_mmm),axis=1)
traf_mmm2['tmja_tv_y']=traf_mmm2.apply(lambda x : somme_traf(x['idtronc_y'], traf_mmm),axis=1)

traf_mmm2['traf_max']=traf_mmm2.apply(lambda x:trafic_reference_mmm(x['tmjo_2_sens_x'],x['tmjo_2_sens_y'],traf_mmm2,x['idtronc_x'],x['idtronc_y']), axis=1)
traf_mmm2.drop_duplicates(['idtronc_x','idtronc_y','tmja_tv_x','tmja_tv_y','traf_max'], inplace=True)

In [123]:
trafc_rens2=trafc_rens.copy()
trafc_rens2['idtronc']=trafc_rens2.apply(lambda x : x['idtronc_x'] if x['tmjo_2_sens_x']==-99
                                                  else x['idtronc_y'], axis=1)
trafc_rens2['ident_ref_calcul']=trafc_rens2.apply(lambda x : 'x' if x['tmjo_2_sens_y']==-99
                                                      else 'y', axis=1)

In [ ]:
#si plusieurs resultas possibles pour unident on garde le max
trafc_rens2=trafc_rens2.loc[trafc_rens2.apply(lambda x : x['traf_max']==x['tmja_tv_'+x['ident_ref_calcul']], axis=1)].copy()

try : 
    trafc_rens2['tmjo_2_sens_extrapol']=trafc_rens2.apply(lambda x : 
        int(x['tmjo_2_sens_x']/x['tmja_tv_x']*x['tmja_tv_y']) if x['tmjo_2_sens_y']==-99 else int(x['tmjo_2_sens_y']/x['tmja_tv_y']*x['tmja_tv_x']),
                                                      axis=1)
    #check si une seule valeur par idtronc que l'on cherche, si pas le cas on prend la valeur calculee max
    if not (trafc_rens2.groupby('idtronc').tmjo_2_sens_extrapol.nunique()==1).all() : 
        trafc_rens2=trafc_rens2.loc[trafc_rens.tmjo_2_sens_extrapol==trafc_rens.groupby('idtronc').tmjo_2_sens_extrapol.transform(max)].copy()   

except (ZeroDivisionError, ValueError) : 
    raise PasDeTraficError (trafc_rens.idtronc.tolist())

In [128]:
traf_mmm2

,ident_x,ident_y,idtronc_x,idtronc_y,tmjo_2_sens_x,tmjo_2_sens_y,cat_rhv_x,cat_rhv_y,NO_x,NO_y,idtronc_a_estim,diff_cat,tmja_tv_x,tmja_tv_y,traf_max
0,37809,37803,700.0,557.0,34330.0,-99.0,1,1,724099606.0,724099596.0,557.0,0,42586.17,44385.40,42586.17
3,37803,37808,557.0,700.0,-99.0,34330.0,1,1,724099596.0,724099602.0,557.0,0,44385.40,42586.17,42586.17


In [120]:
274 in dico_erreur.keys()

True

# ***tests***

In [52]:
idtronc123_pct_sur_perm.idtronc.unique()

array([  710.,  6533.,  4633.,   725.,  1779.,  1625.,  3664., 13252.,
        1630.,  4164.,  1490.,  1656.,  1207.,  1279.,   775.,  2573.,
        1799.,   678.,   949.,  2097.,  2526.,   820.,  3063.,  1032.,
         814.,  1293.,   661.,   975.,  3700.,  5302.,  2402.,  3018.,
        2393.,  3931.,  1146.,   697.,  2433.,  1397.,  4117.,  3401.])

In [ ]:
gdf_rhv_cpt_perm_123=gdf_rhv_cpt_perm_123.reset_index().set_index('ident')
for idtronc_test in sorted(pct_libre_123_avec_dbl.idtronc.unique()) : 
    #récupérer l'idtronc toute cat_rhv des compteurs avec doublons et faire la jointure du trafic sur les lignes concernees
    pct_libre_123_avec_dbl_decompose=lgn_proche_ponct_attr_sup.loc[lgn_proche_ponct_attr_sup['ident'].isin(
        [a for b in pct_libre_123_avec_dbl.loc[pct_libre_123_avec_dbl['idtronc']==idtronc_test].
         id_cpt_2_sens.tolist() for a in b])][['idtronc_tt_rhv','tmjo_2_sens','id_cpt_2_sens']].copy()
    traf_max=pct_libre_123_avec_dbl_decompose.tmjo_2_sens.max()
    id_cpt=pct_libre_123_avec_dbl_decompose.loc[pct_libre_123_avec_dbl_decompose['tmjo_2_sens']==traf_max].id_cpt_2_sens.unique()[0]
    print(id_cpt)
    gdf_rhv_cpt_perm_123.loc[gdf_rhv_cpt_perm_123['idtronc']==idtronc_test,'tmjo_2_sens']=traf_max
    gdf_rhv_cpt_perm_123.loc[gdf_rhv_cpt_perm_123['idtronc']==idtronc_test,'id_cpt_2_sens']=','.join(str(id_cpt))
    gdf_rhv_cpt_perm_123.loc[gdf_rhv_cpt_perm_123['idtronc']==idtronc_test,'type_cpt']='ponctuel'

In [117]:
traf_max=pct_libre_123_avec_dbl_decompose.tmjo_2_sens.max()
id_cpt=pct_libre_123_avec_dbl_decompose.loc[pct_libre_123_avec_dbl_decompose['tmjo_2_sens']==traf_max].id_cpt_2_sens.unique()[0]

(335, 336)

In [156]:
gdf_rhv_cpt_perm_123.loc[528]

,id_x,ident,domanial,groupe,cat_dig,cat_rhv,passage,rggraph_nd,rggraph_na,rgraph_dbl,numero,cdate,mdate,id_ign,nature,sens,codevoie_d,importance,id_y,geometry,source,target,tmjo_2_sens,id_cpt_2_sens,type_cpt
idtronc,,,,,,,,,,,,,,,,,,,,,,,,,
528.0,65175.0,45622,4,1.0,2,2,None,37109.0,23966.0,0.0,Avenue Marcel Dassault,20130102153524,20151022094448,TRONROUT45622,Route Ã 2 chaussÃ©es,Direct,NR,2,TRONROUT45622,"LINESTRING Z (406223.330 6423342.280 0.000, 40...",2712.0,2713.0,NaN,NaN,NaN
528.0,84860.0,52216,4,1.0,2,2,None,52328.0,23968.0,1.0,Avenue Marcel Dassault,20190109083100,20190109083100,TRONROUT52216,Route Ã 1 chaussÃ©e,Double,NR,2,TRONROUT52216,"LINESTRING Z (406406.970 6423349.510 0.000, 40...",3095.0,3096.0,NaN,NaN,NaN
528.0,78851.0,50499,4,1.0,2,2,None,47392.0,47393.0,1.0,Avenue Marcel Dassault,20160905104752,20180502080200,TRONROUT50499,Route Ã 1 chaussÃ©e,Double,NR,2,TRONROUT50499,"LINESTRING Z (407695.390 6423465.180 0.000, 40...",5420.0,5421.0,NaN,NaN,NaN
528.0,78866.0,50508,4,1.0,2,2,None,47408.0,47390.0,1.0,Avenue Marcel Dassault,20160905135232,20180502080208,TRONROUT50508,Route Ã 1 chaussÃ©e,Double,NR,2,TRONROUT50508,"LINESTRING Z (407441.340 6423392.780 0.000, 40...",5424.0,5425.0,NaN,NaN,NaN
528.0,54018.0,39551,4,1.0,2,2,None,30544.0,23375.0,0.0,Avenue Marcel Dassault,20050719155400,20151022094448,TRONROUT39551,Route Ã 2 chaussÃ©es,Direct,NR,2,TRONROUT39551,"LINESTRING Z (407151.220 6423411.170 0.000, 40...",6825.0,6826.0,NaN,NaN,NaN
528.0,36377.0,45623,4,1.0,2,2,None,23967.0,37109.0,0.0,Avenue Marcel Dassault,19951211150200,20151022094448,TRONROUT45623,Route Ã 2 chaussÃ©es,Direct,NR,2,TRONROUT45623,"LINESTRING Z (406244.420 6423327.220 0.000, 40...",8321.0,2712.0,NaN,NaN,NaN
528.0,36381.0,46390,4,1.0,2,2,None,29894.0,23967.0,0.0,Avenue Marcel Dassault,19951211150400,20170217111844,TRONROUT46390,Route Ã 2 chaussÃ©es,Direct,NR,2,TRONROUT46390,"LINESTRING Z (406227.490 6423304.130 0.000, 40...",8322.0,8321.0,NaN,NaN,NaN
528.0,78848.0,50498,4,1.0,2,2,None,47392.0,47393.0,1.0,Avenue Marcel Dassault,20160905104720,20180502080200,TRONROUT50498,Route Ã 1 chaussÃ©e,Double,NR,2,TRONROUT50498,"LINESTRING Z (407695.390 6423465.180 0.000, 40...",5420.0,5421.0,NaN,NaN,NaN
528.0,78850.0,50497,4,1.0,2,2,None,47393.0,47391.0,1.0,Avenue Marcel Dassault,20160905104728,20190913134757,TRONROUT50497,Route Ã 1 chaussÃ©e,Double,NR,2,TRONROUT50497,"LINESTRING Z (407733.060 6423473.660 0.000, 40...",5421.0,8879.0,NaN,NaN,NaN


In [98]:
pct_libre_123_ss_dbl

,idtronc,tmjo_2_sens,id_cpt_2_sens,type_cpt
0,3165.0,1613,"(1050, 1051)",ponctuel
2,7187.0,3620,"(1119,)",ponctuel
3,3139.0,5358,"(1131,)",ponctuel
4,1407.0,10655,"(609, 610)",ponctuel
6,5304.0,16164,"(1582,)",ponctuel
7,1394.0,8419,"(296, 297)",ponctuel
8,2792282.0,7326,"(1903,)",ponctuel
9,1403.0,2133,"(1893, 1894)",ponctuel
11,5657.0,4412,"(904, 905)",ponctuel
13,3520.0,6747,"(1877, 1878)",ponctuel
